In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
user_n_business = pd.read_csv('dataset/yelp_academic_dataset_tip_reduced_100.csv')

In [3]:
business_categories = pd.read_csv('dataset/yelp_academic_dataset_business_100.csv')

In [18]:
business_categories.head()

,business_id,Refinishing Services,Dentists,Drugstores,Massage,Contractors,Motorcycle Dealers,Pets,Hair Removal,Wedding Chapels,...,Pet Sitting,Mexican,Day Spas,Auto Repair,American (Traditional),Chicken Wings,Sandwiches,Financial Services,Automotive,Electronics
0,Apn5Q_b6Nz61Tq4XzPdf9A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AjEbIBw6ZFfln7ePHha9PA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,O8S5hYJ1SMc8fA4QBtVujA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,bFzdJJ3wp3PZssNEsyU23g,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8USyCYqpScwiNEb58Bt6CA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Defining new dataframe

The new dataframe will be a data set that relates users with how many interactions they have with each category.

In [52]:
index = set(user_n_business['user_id']) # getting only unique user ids

In [53]:
columns = business_categories.columns
columns = columns.drop(['business_id']) # take out 'business_id'

In [54]:
df = pd.DataFrame(index=index, columns=columns)

In [55]:
df = df.fillna(0) #Fill NaNs with 0s

In [56]:
df.head()

,Refinishing Services,Dentists,Drugstores,Massage,Contractors,Motorcycle Dealers,Pets,Hair Removal,Wedding Chapels,Churches,...,Pet Sitting,Mexican,Day Spas,Auto Repair,American (Traditional),Chicken Wings,Sandwiches,Financial Services,Automotive,Electronics
OVpt8ITCEWyYBjn0N3CRwg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ulQ8Nyj7jCUR8M83SUMoRQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
o9fbX0eNRqaX0K-PmuXWnQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M5n8mMFoeXjnJEV2gxyGGA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
QGgWWhEi5R4SLAKN-xwtNQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Filling new dataframe with the sum of interactions

In [139]:
business_categories.at[2, 'business_id'] = 'tJRDll5yqpZwehenzE2cSg'

In [143]:
for index, row in user_n_business.iterrows():
    user_id = str(row['user_id'])
    business_id = str(row['business_id'])
    business_index = business_categories.index[business_categories['business_id'] == business_id].tolist()
#     categories_interacted_tosum = business_categories.loc[business_index].drop(['business_id'])
    if business_index:
        print("found business " + business_id + " with index: " + str(business_index[0]))
    else:
        print("coudnt find business with id: " + business_id)
    
    # Increment user interaction with each category of the business
#     new_value = df.loc['OVpt8ITCEWyYBjn0N3CRwg'][" Sandwiches"] + business_categories.loc[business_index][" Sandwiches"]
#     df.at['OVpt8ITCEWyYBjn0N3CRwg', " Sandwiches"] = new_value
#     print(df.loc['OVpt8ITCEWyYBjn0N3CRwg'][" Sandwiches"])

found business tJRDll5yqpZwehenzE2cSg with index: 2
coudnt find business with id: 2ThtBbeDqFkVi6LugUOcVA
coudnt find business with id: jH19V2I9fIslnNhDzPmdkA
coudnt find business with id: dAa0hB2yrnHzVmsCkN4YvQ
coudnt find business with id: dAa0hB2yrnHzVmsCkN4YvQ
coudnt find business with id: ESzO3Av0b1_TzKOiqzbQYQ
coudnt find business with id: k7WRPbDd7rztjHcGGkEjlw
coudnt find business with id: k7WRPbDd7rztjHcGGkEjlw
coudnt find business with id: SqW3igh1_Png336VIb5DUA
coudnt find business with id: KNpcPGqDORDdvtekXd348w
coudnt find business with id: KNpcPGqDORDdvtekXd348w
coudnt find business with id: KNpcPGqDORDdvtekXd348w
coudnt find business with id: 8qNOI6Q1-rJrvWWD5Btz6w
coudnt find business with id: 8qNOI6Q1-rJrvWWD5Btz6w
coudnt find business with id: A73Dp0lo9s_Ci9NFDqbhqw
coudnt find business with id: --ujyvoQlwVoBgMYtADiLA
coudnt find business with id: FCUVjQf762no86Uzcbv1Tg
coudnt find business with id: Tl_jT2a0bRMac5_YW65GPg
coudnt find business with id: Tl_jT2a0bRMac5_YW

In [134]:
df.head()

,Refinishing Services,Dentists,Drugstores,Massage,Contractors,Motorcycle Dealers,Pets,Hair Removal,Wedding Chapels,Churches,...,Pet Sitting,Mexican,Day Spas,Auto Repair,American (Traditional),Chicken Wings,Sandwiches,Financial Services,Automotive,Electronics
OVpt8ITCEWyYBjn0N3CRwg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,110,0,0,0
ulQ8Nyj7jCUR8M83SUMoRQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
o9fbX0eNRqaX0K-PmuXWnQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M5n8mMFoeXjnJEV2gxyGGA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
QGgWWhEi5R4SLAKN-xwtNQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
business_categories.head()

,business_id,Refinishing Services,Dentists,Drugstores,Massage,Contractors,Motorcycle Dealers,Pets,Hair Removal,Wedding Chapels,...,Pet Sitting,Mexican,Day Spas,Auto Repair,American (Traditional),Chicken Wings,Sandwiches,Financial Services,Automotive,Electronics
0,Apn5Q_b6Nz61Tq4XzPdf9A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AjEbIBw6ZFfln7ePHha9PA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,O8S5hYJ1SMc8fA4QBtVujA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,bFzdJJ3wp3PZssNEsyU23g,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8USyCYqpScwiNEb58Bt6CA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
business_categories.loc[[2],' Sandwiches']

2    1
Name:  Sandwiches, dtype: int64

In [100]:
user_n_business.head()

,user_id,business_id
0,zcTZk7OG8ovAmh_fenH21g,tJRDll5yqpZwehenzE2cSg
1,M5n8mMFoeXjnJEV2gxyGGA,2ThtBbeDqFkVi6LugUOcVA
2,ZcLKXikTHYOnYt5VYRO5sg,jH19V2I9fIslnNhDzPmdkA
3,oaYhjqBbh18ZhU0bpyzSuw,dAa0hB2yrnHzVmsCkN4YvQ
4,ulQ8Nyj7jCUR8M83SUMoRQ,dAa0hB2yrnHzVmsCkN4YvQ
